>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Getting_Started)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Getting_Started) to leverage the power of whylogs and WhyLabs together!*

# Getting Started

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/basic/Getting_Started.ipynb)

## Table of Content

In this example, we'll explore the basics of logging data with whylogs and a user defined function or UDFs

## Installing whylogs

In [ ]:
%pip install whylogs

## Loading a Pandas DataFrame

Before showing how we can log data, we first need the data itself. Let's create a simple Pandas DataFrame:

In [1]:
import pandas as pd
data = {
    "animal": ["cat", "hawk", "snake", "cat", "mongoose"],
    "legs": [4, 2, 0, 4, 4],
    "weight": [4.3, 1.8, 1.3, 4.1, 5.4],
}

df = pd.DataFrame(data)


In [2]:
import whylogs as why
from whylogs.experimental.core.udf_schema import udf_schema
from whylogs.experimental.core.metrics.udf_metric import register_metric_udf


@register_metric_udf(col_name="animal")
def has_cool_animal_name(text):
  if len(text) > 4: # long names are cool
    return 1
  else:
    return 0
  
custom_schema = udf_schema()

## Profiling with whylogs

To obtain a profile of your data, you can simply use whylogs' `log` call with your UDF schema defined earlier. This will attach a feature named `animal.has_cool_animal_name` which you can then see in WhyLabs.

In [ ]:
import whylogs as why

why.init()

results = why.log(df, name="udf_demo", schema=custom_schema)
results.view().to_pandas()